In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import VarianceThreshold,mutual_info_classif,SelectKBest,SelectFromModel

import lightgbm as lgb

from sklearn.model_selection import  GridSearchCV,cross_val_score,TimeSeriesSplit,cross_validate
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from sklearn import tree
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, recall_score, precision_score, roc_auc_score,roc_curve, auc,matthews_corrcoef,f1_score,brier_score_loss,make_scorer
 
from sklearn.decomposition import PCA

/Users/neenu/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#reading csv files
X_train=pd.read_csv('X_train_F.csv')
Y_train=pd.read_csv('Y_train_F.csv')
X_test=pd.read_csv('X_test_F.csv')
Y_test=pd.read_csv('Y_test_F.csv')

In [3]:
X_Data=pd.read_csv('X_Data.csv')
Y_Data=pd.read_csv('Y_Data.csv')

In [4]:
X_train= X_train[:70000]
Y_train=Y_train[:70000]

In [5]:
# with default paramter
clf = lgb.LGBMClassifier(class_weight="balanced")
pca = PCA(.96)
pipeline_lgbm  = Pipeline([('feature_sele',pca),('clf',clf)])

In [6]:
pipeline_lgbm.fit(X_train,Y_train.values.ravel())
y_trainpred=pipeline_lgbm.predict_proba(X_train)
y_testpred = pipeline_lgbm.predict_proba(X_test)

In [7]:
# auc with default parameters
print('train auc with default parameter:',roc_auc_score(Y_train,y_trainpred[:,1]))
print('test auc with default parameter:',roc_auc_score(Y_test,y_testpred[:,1]))

train auc with default parameter: 0.9579975474720932
test auc with default parameter: 0.38430938761257705


In [8]:
print('brier_score_loss:',brier_score_loss(Y_test,y_testpred[:,1]))

brier_score_loss: 0.1562250286772807


In [9]:
y_testpred_ =pipeline_lgbm.predict(X_test)
print("Recall:\n",metrics.recall_score(Y_test,y_testpred_))
print("matthews_corrcoef:\n",matthews_corrcoef(Y_test, y_testpred_))

Recall:
 0.07482337829158639
matthews_corrcoef:
 -0.025826502116423213


In [12]:
#  parameters currently used
print('Parameters currently in use:\n')
print(clf.get_params())

Parameters currently in use:

{'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


In [13]:
#GridSearchCV with timeseries

time_splitter_inner = TimeSeriesSplit(n_splits = 3)
time_splitter_outer= TimeSeriesSplit(n_splits = 10)

model=lgb.LGBMClassifier(class_weight="balanced")
params = {"max_depth": [5,10,15],
              "learning_rate" : [0.1,0.15,0.3],
              "num_leaves": [32,150,200],
              "n_estimators": [500,700],
              'boost_from_average': [False],
               "random_state": [2]}
pca = PCA(.96)

gs_lgbm = GridSearchCV(model, param_grid = params, cv = time_splitter_inner, scoring = 'roc_auc',verbose=10, n_jobs=-1)

pipelinelgbm  = Pipeline([('feature_sele',pca),
                      ('clf_cv',gs_lgbm)])

In [14]:
pipelinelgbm .fit(X_Data,Y_Data.values.ravel())

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  4.3min finished


Pipeline(steps=[('feature_sele', PCA(n_components=0.96)),
                ('clf_cv',
                 GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                              estimator=LGBMClassifier(class_weight='balanced'),
                              n_jobs=-1,
                              param_grid={'boost_from_average': [False],
                                          'learning_rate': [0.1, 0.15, 0.3],
                                          'max_depth': [5, 10, 15],
                                          'n_estimators': [500, 700],
                                          'num_leaves': [32, 150, 200],
                                          'random_state': [2]},
                              scoring='roc_auc', verbose=10))])

In [15]:
custom_scorer = {
                 'recall': make_scorer(recall_score),
                 'Brier': make_scorer(brier_score_loss),
                 'Matthewscorrcoef': make_scorer(matthews_corrcoef),
                 'ROC_AUC': make_scorer(roc_auc_score)
                 }
lgbm_scores = cross_validate(pipelinelgbm,X_Data,Y_Data.values.ravel(),cv = time_splitter_outer, scoring = custom_scorer)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:   36.5s finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  1.2min finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  3.3min finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  3.5min finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   48.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  4.0min finished


In [16]:
sorted(lgbm_scores.keys())

['fit_time',
 'score_time',
 'test_Brier',
 'test_Matthewscorrcoef',
 'test_ROC_AUC',
 'test_recall']

In [17]:
lgbm_time = lgbm_scores['fit_time']
lgbm_recall = lgbm_scores['test_recall']
lgbm_Brier = lgbm_scores['test_Brier']
lgbm_Matthewscorrcoef = lgbm_scores['test_Matthewscorrcoef']
lgbm_ROC_AUC = lgbm_scores['test_ROC_AUC']


In [18]:
print("=== fit time ===")
print(lgbm_time)
print("=== All recall Scores ===")
print(lgbm_recall)
print("=== All Brier Scores ===")
print(lgbm_Brier)
print("=== All Matthewscorrcoef Scores ===")
print(lgbm_Matthewscorrcoef)
print("=== All ROC_AUC Scores ===")
print(lgbm_ROC_AUC)

=== fit time ===
[ 37.26967788  71.26410413 103.40467286 129.36811996 132.60759592
 155.0201993  177.18246794 200.5147779  215.03260112 245.54582   ]
=== All recall Scores ===
[0.10215054 0.02793296 0.09947644 0.19       0.06779661 0.1954023
 0.07207207 0.17475728 0.11965812 0.18232044]
=== All Brier Scores ===
[0.02938865 0.02828854 0.02781707 0.02828854 0.03221751 0.02483105
 0.01885903 0.0466761  0.01854471 0.02420242]
=== All Matthewscorrcoef Scores ===
[0.21348589 0.10731818 0.27465191 0.34948738 0.11202542 0.34869932
 0.14514088 0.17266993 0.23388002 0.38627165]
=== All ROC_AUC Scores ===
[0.54945636 0.51348136 0.54933316 0.59353967 0.5306652  0.59624696
 0.53467647 0.57706518 0.55862829 0.59067494]


In [19]:
print("=== Mean recall Score ===")
print( lgbm_recall.mean())
print("=== Mean Brier Score ===")
print( lgbm_Brier.mean())
print("=== Mean Matthewscorrcoef  Score ===")
print( lgbm_Matthewscorrcoef .mean())
print("=== Mean ROC_AUC  Score ===")
print( lgbm_ROC_AUC .mean())
print("=== Mean fit_time ===")
print( lgbm_time .mean())

=== Mean recall Score ===
0.12315667626114457
=== Mean Brier Score ===
0.02791136256482791
=== Mean Matthewscorrcoef  Score ===
0.23436305770010088
=== Mean ROC_AUC  Score ===
0.5593767589138491
=== Mean fit_time ===
146.72100369930268


In [20]:

print("=== Minimum recall Score ===")
print( lgbm_recall.min())
print("=== Minimum Brier Score ===")
print(  lgbm_Brier.min())
print("=== Minimum Matthewscorrcoef  Score ===")
print(  lgbm_Matthewscorrcoef.min())
print("=== Minimum ROC_AUC  Score ===")
print(  lgbm_ROC_AUC.min())
print("=== Minimum RF_time  ===")
print(  lgbm_time.min())

=== Minimum recall Score ===
0.027932960893854747
=== Minimum Brier Score ===
0.018544711614018546
=== Minimum Matthewscorrcoef  Score ===
0.10731817815108481
=== Minimum ROC_AUC  Score ===
0.5134813575491266
=== Minimum RF_time  ===
37.26967787742615


In [21]:
print("=== Maximum recall Score ===")
print( lgbm_recall.max())
print("=== Minimum Brier Score ===")
print( lgbm_Brier.min())
print("=== Maximum Matthewscorrcoef  Score ===")
print( lgbm_Matthewscorrcoef .max())
print("=== Maximum ROC_AUC  Score ===")
print( lgbm_ROC_AUC .max())
print("=== Maximum fit_time ===")
print( lgbm_time .max())

=== Maximum recall Score ===
0.19540229885057472
=== Minimum Brier Score ===
0.018544711614018546
=== Maximum Matthewscorrcoef  Score ===
0.38627164730894853
=== Maximum ROC_AUC  Score ===
0.5962469565023596
=== Maximum fit_time ===
245.54581999778748
